# Laboratorio 3

En este laboratorio nuevamente utilizaremos los datos de COVID-19 disponibilizados por el Ministerio de Ciencias, Tecnología, Conocimiento e Innovación de Chile ([info](https://github.com/MinCiencia/Datos-COVID19/tree/master/output/producto1)).

In [1]:
import pandas as pd

covid = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto1/Covid-19_std.csv")
covid.head(10)

,Region,Codigo region,Comuna,Codigo comuna,Poblacion,Fecha,Casos confirmados
0,Arica y Parinacota,15,Arica,15101.0,247552.0,2020-03-30,6.0
1,Arica y Parinacota,15,Camarones,15102.0,1233.0,2020-03-30,0.0
2,Arica y Parinacota,15,General Lagos,15202.0,810.0,2020-03-30,0.0
3,Arica y Parinacota,15,Putre,15201.0,2515.0,2020-03-30,0.0
4,Arica y Parinacota,15,Desconocido Arica y Parinacota,NaN,NaN,2020-03-30,NaN
5,Tarapacá,1,Alto Hospicio,1107.0,129999.0,2020-03-30,0.0
6,Tarapacá,1,Camina,1402.0,1375.0,2020-03-30,0.0
7,Tarapacá,1,Colchane,1403.0,1583.0,2020-03-30,0.0
8,Tarapacá,1,Huara,1404.0,3000.0,2020-03-30,0.0
9,Tarapacá,1,Iquique,1101.0,223463.0,2020-03-30,5.0


## Ejercicio 1

(1 pto)

¿Cuál es la región que tiene menos registros (filas)?

In [2]:
nombre = covid.value_counts("Region").loc[covid.value_counts("Region") == covid.value_counts("Region").iloc[-1]].keys()[0]
casos_num = covid.value_counts("Region").iloc[-1]
print(f"La respuesta es: {nombre} con {casos_num} casos.")

La respuesta es: Arica y Parinacota con 275 casos.


__Respuesta:__ La región de Arica y Parinacota

## Ejercicio 2

(2 puntos)

1. Define la serie `casos` tal que:
    * Solo sean los registros de la región Valparaíso.
    * Los _index_ deben ser la _Fecha_ pero en formato `datetime`. Hint: `pd.to_datetime()`
    * Los _values_ deben ser los _Casos confirmados_.
2. Define la serie `casos_not_null` filtrando los calores `NaN` de la serie `casos`.
3. Obtén el mínimo, máximo y suma y promedio de la serie `casos_not_null`.

In [3]:
covid_valpo = covid.query("Region == 'Valparaíso'")
indices = pd.to_datetime(covid_valpo.loc[:,"Fecha"].values)
values = covid_valpo.loc[:,"Casos confirmados"]
casos = pd.Series(values.values, index=indices)
casos.head()

2020-03-30    7.0
2020-03-30    0.0
2020-03-30    6.0
2020-03-30    0.0
2020-03-30    0.0
dtype: float64

In [4]:
casos_not_null = casos.loc[~casos.isnull()]

In [5]:
casos_min = casos_not_null.min()
casos_max = casos_not_null.max()
casos_sum = sum(casos_not_null)
casos_mean = casos_sum/len(casos_not_null)

In [6]:
print(f"Mínimo casos: {casos_min}")
print(f"Máximo casos: {casos_max}")
print(f"Suma de casos: {casos_sum}")
print(f"Promedio de casos: {casos_mean}")

Mínimo casos: 0.0
Máximo casos: 7020.0
Suma de casos: 790461.0
Promedio de casos: 373.2110481586402


## Ejercicio 3

* Define el dataframe  `covid_full` uniendo (`merge`) los dataframes `covid_total` y `covid_death` utilizando la _Fecha_ como _key_ y de tal forma de unir solo los registros que se encuentren en ambos conjuntos de datos. Hint: Escoger sabiamente entre _left_, _right_, _inner_ u _outer join_.
* Filtra el dataframe `covid_full` tal que:
    - Fechas con por lo menos 1000 casos totales.
    - Fechas con por lo menos 100 casos nuevos con síntomas.
    - Fechas con más de 20 muertes en el grupo etárea `70-79`.

¿Cuál es la fecha máxima y mínima? ¿Cuántos registros quedan finalmente?

In [7]:
covid_total = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto5/TotalesNacionales_T.csv", usecols=range(3))
covid_total.head()

,Fecha,Casos nuevos con sintomas,Casos totales
0,2020-03-02,1.0,1.0
1,2020-03-03,0.0,1.0
2,2020-03-04,2.0,3.0
3,2020-03-05,1.0,4.0
4,2020-03-06,1.0,5.0


In [8]:
covid_death = pd.read_csv("https://raw.githubusercontent.com/MinCiencia/Datos-COVID19/master/output/producto10/FallecidosEtario_T.csv").rename(columns={"Grupo de edad": "Fecha"})
covid_death.head()

,Fecha,<=39,40-49,50-59,60-69,70-79,80-89,>=90
0,2020-04-09,2,1,1,9,21,19,4
1,2020-04-10,2,2,2,10,22,21,6
2,2020-04-11,3,2,2,12,24,24,6
3,2020-04-12,3,2,3,12,28,25,7
4,2020-04-13,3,2,3,12,29,26,7


In [9]:
aux = pd.merge(covid_total, covid_death, how="inner")
covid_full = pd.DataFrame(aux.drop("Fecha", axis=1).values, index=aux["Fecha"].values, columns = aux.drop("Fecha", axis=1).columns)
covid_full_filtered = covid_full.loc[lambda x: (x["Casos totales"]>=1000) & (x["Casos nuevos con sintomas"]>=100) & (x["70-79"]>20)]
covid_full_filtered

,Casos nuevos con sintomas,Casos totales,<=39,40-49,50-59,60-69,70-79,80-89,>=90
2020-04-09,426.0,5972.0,2.0,1.0,1.0,9.0,21.0,19.0,4.0
2020-04-10,529.0,6501.0,2.0,2.0,2.0,10.0,22.0,21.0,6.0
2020-04-11,426.0,6927.0,3.0,2.0,2.0,12.0,24.0,24.0,6.0
2020-04-12,286.0,7213.0,3.0,2.0,3.0,12.0,28.0,25.0,7.0
2020-04-13,312.0,7525.0,3.0,2.0,3.0,12.0,29.0,26.0,7.0
...,...,...,...,...,...,...,...,...,...
2020-09-28,1112.0,459671.0,281.0,430.0,1283.0,2687.0,3606.0,3241.0,1170.0
2020-09-29,1036.0,461300.0,281.0,431.0,1286.0,2695.0,3609.0,3248.0,1175.0
2020-09-30,1185.0,462991.0,281.0,432.0,1287.0,2699.0,3613.0,3250.0,1179.0
2020-10-01,1198.0,464750.0,284.0,435.0,1296.0,2710.0,3645.0,3264.0,1188.0


In [10]:
print(f"Fecha mínima: {covid_full_filtered.index.min()}")
print(f"Fecha máxima: {covid_full_filtered.index.max()}")
print(f"Cantidad de registros finalmente: {len(covid_full_filtered.index)}")

Fecha mínima: 2020-04-09
Fecha máxima: 2020-10-02
Cantidad de registros finalmente: 177
